In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers-ensemble/cat_v2_submission.csv
/kaggle/input/lg-aimers-ensemble/sample_submission.csv
/kaggle/input/lg-aimers-ensemble/nn_v2_oof_val_predictions.csv
/kaggle/input/lg-aimers-ensemble/cat_v2_oof_val_predictions.csv
/kaggle/input/lg-aimers-ensemble/lgbm_baseline_oof_test_predictions.csv
/kaggle/input/lg-aimers-ensemble/cat_v2_oof_test_predictions.csv
/kaggle/input/lg-aimers-ensemble/lgbm_baseline_submission.csv
/kaggle/input/lg-aimers-ensemble/lgbm_baseline_oof_val_predictions.csv
/kaggle/input/lg-aimers-ensemble/v2_nn_submission.csv
/kaggle/input/lg-aimers-ensemble/nn_v2_oof_test_predictions.csv
/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [2]:
cat_val = pd.read_csv('/kaggle/input/lg-aimers-ensemble/cat_v2_oof_val_predictions.csv', index_col = 0)
lgbm_val = pd.read_csv('/kaggle/input/lg-aimers-ensemble/lgbm_baseline_oof_val_predictions.csv', index_col = 0)
nn_val = pd.read_csv('/kaggle/input/lg-aimers-ensemble/nn_v2_oof_val_predictions.csv', index_col = 0)

In [3]:
lgbm_preds = lgbm_val["oof_val_preds"].values
cat_preds = cat_val["oof_val_preds"].values
nn_preds = nn_val['oof_val_preds'].values

In [4]:
train = pd.read_csv('/kaggle/input/lg-aimers/train.csv', index_col= 0)
y = train['임신 성공 여부'].values
test = pd.read_csv('/kaggle/input/lg-aimers/test.csv', index_col= 0)

In [5]:
from sklearn.metrics import roc_auc_score

In [6]:
print('LGBM OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, lgbm_preds))

LGBM OOF 검증 데이터 ROC-AUC : 0.7404986919647051


In [7]:
print('CatBoost OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, cat_preds))

CatBoost OOF 검증 데이터 ROC-AUC : 0.7399599375435523


In [8]:
print('NN OOF 검증 데이터 ROC-AUC :', roc_auc_score(y, nn_preds))

NN OOF 검증 데이터 ROC-AUC : 0.7376636074816705


In [9]:
ensemble_preds = lgbm_preds * 0.6 + cat_preds * 0.25 + nn_preds * 0.15

ensemble_roc_auc = roc_auc_score(y, ensemble_preds)
print(f"앙상블 OOF 검증 데이터 ROC-AUC: {ensemble_roc_auc}")

앙상블 OOF 검증 데이터 ROC-AUC: 0.740735142628162


In [10]:
submission = pd.read_csv('/kaggle/input/lg-aimers/sample_submission.csv', index_col= 'ID')

In [11]:
cat_test = pd.read_csv('/kaggle/input/lg-aimers-ensemble/cat_v2_oof_test_predictions.csv', index_col = 0)
lgbm_test = pd.read_csv('/kaggle/input/lg-aimers-ensemble/lgbm_baseline_oof_test_predictions.csv', index_col = 0)
nn_test = pd.read_csv('/kaggle/input/lg-aimers-ensemble/nn_v2_oof_test_predictions.csv', index_col = 0)

In [12]:
lgbm_test_preds = lgbm_test["oof_test_preds"].values
cat_test_preds = cat_test["oof_test_preds"].values
nn_test_preds = nn_test['oof_test_preds'].values

In [13]:
ensemble_test_preds = lgbm_test_preds * 0.6 + cat_test_preds * 0.25 + nn_test_preds * 0.15

In [14]:
submission["probability"] = ensemble_test_preds
submission.to_csv("lgbm_cat_nn_submission.csv", index=True)

In [15]:
submission

,probability
ID,
TEST_00000,0.001524
TEST_00001,0.004749
TEST_00002,0.156787
TEST_00003,0.105011
TEST_00004,0.504799
...,...
TEST_90062,0.001556
TEST_90063,0.318149
TEST_90064,0.493329


In [17]:
# OOF 검증 예측 결과 저장
oof_val_preds_df = pd.DataFrame({'ID': train.index, 'oof_val_preds': ensemble_preds})
oof_val_preds_df.to_csv("ensemble_oof_val_predictions.csv", index=True)

# OOF 테스트 예측 결과 저장
oof_test_preds_df = pd.DataFrame({'ID': test.index, 'oof_test_preds': ensemble_test_preds})
oof_test_preds_df.to_csv("ensemble_oof_test_predictions.csv", index=True)